In [92]:
import requests
import os
import json
from bs4 import BeautifulSoup
from urllib.parse import urlparse

import mysql.connector

In [37]:
from pyLightspeed.lsretail import api as lsretail
from pyLightspeed.lsecom import api as lsecom

In [94]:

#establishing the connection
conn = mysql.connector.connect(
   user='jamie', password='W!neL0ver', host='127.0.0.1', database='vintagewine')

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [38]:
KEY_FILE = "D:\Development\.keys\lightspeed_keys.json"

with open(KEY_FILE) as f:
    keys = json.load(f)

store_data = {
            'account_id': keys["account_id"],
            'save_path': 'D:\\Development\\.keys\\'
            }

credentials = {
            'client_id': keys["client_id"],
            'client_secret': keys["client_secret"],
            'api_key': keys["api_key"],
            'api_secret': keys["api_secret"]
            }

In [39]:
lsr = lsretail.Connection(store_data, credentials)

In [40]:
page_link = 'https://www.wine.com/product/ridge-pagani-ranch-zinfandel-2016/510360'
item_id = '994'
system_id = '210000000998'

image_path = 'D:\\Data\\'
# this is the url that we've already determined is safe and legal to scrape from.
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}

In [41]:

page_response = requests.get(page_link, headers=headers, timeout=5)
# here, we fetch the content from the url, using the requests library
page_content = BeautifulSoup(page_response.content, "html.parser")
#we use the html parser to parse the url content and store it in a variable.
textContent = []


In [42]:
def process_image(url, image_path, filename, item_id, description = "Image", ordering = 0):
    with open(image_path+filename, "wb") as f:
        f.write(requests.get(url).content)
    url = lsr.api_url+'Image.json'
    
    files = {'image': (filename, open(image_path + filename, 'rb'), 'image/jpeg')}
    payload = {'data': '{"description": "' + description + '", "ordering": ' + ordering +', "itemID": ' + item_id +'}'}
    r = requests.post(url, files=files, data=payload, headers=lsr.headers)
    print(r.text)

# Get Content
Get's the content from the page


In [45]:
description = page_content.find(class_='viewMoreModule_text')
description_text = page_content.find(class_='viewMoreModule_text').text

In [35]:
page_content.select('body > div.corePage.winePage > main > section.pipSecContent > section.pipSecContent_topSection > div.pipWineNotes > div > div.viewMoreModule_text')


[]

In [67]:
reviews = page_content.select('div.pipProfessionalReviews_list')
reviews

[<div class="pipProfessionalReviews_list"><div class="pipProfessionalReviews_rating"><span class="wineRatings_initials">V</span> <span class="wineRatings_rating">94</span></div><div class="pipProfessionalReviews_authorName">Vinous</div><div class="pipProfessionalReviews_review"><div class="pipSecContent_copy">The 2016 Zinfandel Pagani Ranch is bright, focused and beautifully perfumed. Bright red cherry, blood orange, pomegranate, mint and rose petal notes are all lifted in this exquisite, finessed Zinfandel. Silky, medium in body and gracious, the 2016 is absolutely sublime. The oak is just a bit pronounced at this stage, but that should not be an issue over time.</div></div></div>,
 <div class="pipProfessionalReviews_list"><div class="pipProfessionalReviews_rating"><span class="wineRatings_initials">RP</span> <span class="wineRatings_rating">93</span></div><div class="pipProfessionalReviews_authorName">Robert Parker's Wine Advocate</div><div class="pipProfessionalReviews_review"><div 

In [85]:
reviews[1].select('span[class="wineRatings_initials"]')

[<span class="wineRatings_initials">RP</span>]

In [106]:
# This is an attempt to get stuf to write in to the DB, but it failed
for review in reviews:
    initials = review.select('span[class="wineRatings_initials"]')[0].text
    rating = review.select('span[class="wineRatings_rating"]')[0].text
    author = review.select('div[class="pipProfessionalReviews_authorName"]')[0].text
    review_detail = review.select('div[class="pipProfessionalReviews_review"]')[0].text
    review_html = review.select('div[class="pipProfessionalReviews_review"]')[0]
   
    
    sql = "INSERT INTO wine_reviews(source, initials, review_detail, review_html, wine_itemID, wine_systemSKU, url) VALUES ('%(author)s', '%(initials)s', '%(review_detail)s', '%(review_html)s', %(item_id)s, %(system_id)s, '%(page_link)s')"
    data = {
        'author': author,
        'initials': initials,
        'review_detail': review_detail,
        'review_html': review_html, 
        'item_id': item_id,
        'system_id': system_id,
        'page_link': page_link
        }
    
    cursor.execute(sql, data)




MySQLInterfaceError: Python type Tag cannot be converted

In [ ]:
 
# This is an attempt to get stuf to write in to the DB, but it failed
for review in reviews:
    initials = review.select('span[class="wineRatings_initials"]')[0].text
    rating = review.select('span[class="wineRatings_rating"]')[0].text
    author = review.select('div[class="pipProfessionalReviews_authorName"]')[0].text
    review_detail = review.select('div[class="pipProfessionalReviews_review"]')[0].text
    review_html = review.select('div[class="pipProfessionalReviews_review"]')[0]
   
    
    sql = "INSERT INTO wine_reviews(source, initials, review_detail, review_html, wine_itemID, wine_systemSKU, url) VALUES ('%(author)s', '%(initials)s', '%(review_detail)s', '%(review_html)s', %(item_id)s, %(system_id)s, '%(page_link)s')"
    data = {
        'author': author,
        'initials': initials,
        'review_detail': review_detail,
        'review_html': review_html, 
        'item_id': item_id,
        'system_id': system_id,
        'page_link': page_link
        }
    
    cursor.execute(sql, data)




# Do the Images
Gets the wine images off the page, catches all the information, stores it, and puts the image in Lightspeed via API

In [8]:
# images=page_content.select('body > div.corePage.winePage > main > section.pipMainContent > div.pipHero.prodImageZoom.js-thumbs-enabled > div.pipHero_wrapper > div > picture > img')
# images = page_content.get('img')
images = page_content.find_all(class_='pipThumb_image')
images

[<img alt="Ridge Pagani Ranch Zinfandel 2016 Front Label" class="pipThumb_image pipThumb_img-0" src="/product/images/w_75,h_75,c_pad,fl_progressive/mlmzi1ovp1hwnj6zaqtn.jpg"/>,
 <img alt="Ridge Pagani Ranch Zinfandel 2016  Front Bottle Shot" class="pipThumb_image pipThumb_img-1" src="/product/images/w_75,h_75,c_pad,fl_progressive/dtdjzop7jqszgwspquu8.jpg"/>]

In [13]:
for image in images:
    # urlparse will take the url from the image link...
    a = urlparse(image.get('src'))
    # Then we split out the filename and the extension
    (sourcefilename, ext) = os.path.splitext(os.path.basename(a.path))
    link = 'https://www.wine.com/product/images/'+ os.path.basename(a.path)
    ordering = "0"

    if "Bottle" in image.get('alt'):
        filename = system_id + '_btl'+ ext
        ordering = "1"
    elif "Label" in image.get('alt'):
        filename = system_id + '_lbl'+ ext
        ordering = "2"    
    else:
        filename = system_id + sourcefilename + ext
    process_image(link, image_path, filename, item_id, description = image.get('alt'), ordering = ordering)
    print(f"This is {image.get('alt')} from {a.path} saved as {filename}")
    

{"httpCode":"401","httpMessage":"Unauthorized","message":"Invalid access token.","errorClass":"BadAuthenticationRequestException"}
This is Ridge Pagani Ranch Zinfandel 2016 Front Label from /product/images/w_75,h_75,c_pad,fl_progressive/mlmzi1ovp1hwnj6zaqtn.jpg saved as 210000000998_lbl.jpg
{"httpCode":"401","httpMessage":"Unauthorized","message":"Invalid access token.","errorClass":"BadAuthenticationRequestException"}
This is Ridge Pagani Ranch Zinfandel 2016  Front Bottle Shot from /product/images/w_75,h_75,c_pad,fl_progressive/dtdjzop7jqszgwspquu8.jpg saved as 210000000998_btl.jpg


In [ ]:

url = 'https://api.lightspeedapp.com/API/Account/190211/Image.json'
files = {'image': ('210000001629_btl.jpg', open(image_path+'210000001629_btl.jpg', 'rb'), 'image/jpeg')}
payload = {'data': '{"description": "Ridge Pagani Ranch Zinfandel 2016 Front Label", "ordering": 2, "itemID": ' + item_id +'}'}

r = requests.post(url, files=files, data=payload, headers=lsr.headers)

print(r.text)

In [72]:
# This prints out the document in a formatted tree style and limits the number of characters returned
print(page_content.prettify()[0:2000])

<!DOCTYPE doctype html>
<html class="loading" data-view1="true" data-view1_10="true" data-view1_9="true" lang="en" xml:lang="en">
 <head>
  <meta content="Wine" name="salesChannel"/>
  <meta content="Zinfandel" name="productVarietal"/>
  <meta content="Sonoma Valley" name="productRegion"/>
  <meta content="$48.99" name="productPrice"/>
  <meta content="Sonoma Valley, Sonoma County, California" name="productOrigin"/>
  <meta content="510360" name="productID"/>
  <meta content="In Stock" name="ProductAvailability"/>
  <meta content="SOU913205_2016" name="pProductID"/>
  <meta content="Product Detail" name="pageType"/>
  <meta content="Ridge Pagani Ranch Zinfandel 2016 from Sonoma Valley, Sonoma County, California - Black cherry, pepper and chaparral on the nose. Briary red fruit, well defined, chalky tannins and refreshing acidity on the palate. Long sensuous finish. Since..." name="description"/>
  <meta content="true" name="CartLoaded"/>
  <meta content="" name="PromoCode"/>
  <meta co

In [ ]:
page_content.select('body > div.lightBox > div.lightBoxContentWrap > span.lightBoxContent > img.lightBoxContent_image.lightBoxContent_image-active')

In [ ]:
images[1].get('class')[0]


In [ ]:
a = urlparse(images[0].get('src'))
print(a.path)                    # Output: /kyle/09-09-201315-47-571378756077.jpg
print(os.path.basename(a.path))  # Output: 09-09-201315-47-571378756077.jpg

In [ ]:
link = images[0].attrs
link

# Put the images in Lightspeed Retail

# Descriptions
This is some of the Descriptions

In [ ]:
page_content.select('body > div.corePage.winePage > main > section.pipSecContent > section.pipSecContent_topSection > div.pipWineNotes')

In [ ]:
page_content.select("img[src^=http]")

In [ ]:
for image in images:
        lnk = link["src"]

